# o-vton
## appearance generation
### train

#### 필요한 패키지 설치

In [1]:
import os
import sys

IS_COLAB = False

if 'COLAB_GPU' in os.environ:
    IS_COLAB=True
else:
    IS_COLAB=False

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install dominate
!pip install av
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 36.9MB 149kB/s 
     |████████████████████████████████| 276kB 11.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=dd8a03b8943cd17f79270af0921f3de144cdf046841e2b130604144bda0c4cdd
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# console에서 다음 작업 진행할 것

# !git clone https://github.com/wonyangcho/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data.git /content/o-vton 
# !cd ..

In [5]:
if IS_COLAB == True:
    from google.colab import drive

In [6]:
!nvidia-smi

Sun Dec 27 12:48:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from pathlib import Path

my_file = Path("/content/dataset")
if my_file.is_dir() == False:
  !wget -O o-vton-dataset.zip https://www.dropbox.com/s/3kexhszm15z8j2d/o-vton-dataset.zip?dl=0
  !unzip -o -q o-vton-dataset.zip 
  !rm o-vton-dataset.zip
!ls ./dataset


--2020-12-27 12:48:19--  https://www.dropbox.com/s/q58xuki764dnh06/o-vton-dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q58xuki764dnh06/o-vton-dataset.zip [following]
--2020-12-27 12:48:20--  https://www.dropbox.com/s/raw/q58xuki764dnh06/o-vton-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc926e1728c5c0a24b9b565403e8.dl.dropboxusercontent.com/cd/0/inline/BF1pAPOUglcx2mfH-Kbuz8LpokDMND39wjrvZIqEuiJgwsVE9ZouZb6hCkhioF95r6EZAicEwoUHHsXh1Ow0zetlSBAO_TeRRWDjgYaio3-xZcTKgwWBgFfgTjseJvgbHSA/file# [following]
--2020-12-27 12:48:20--  https://uc926e1728c5c0a24b9b565403e8.dl.dropboxusercontent.com/cd/0/inline/BF1pAPOUglcx2mfH-Kbuz8LpokDMND39wjrvZIqEuiJgwsVE9ZouZb6hCkhioF95r6EZAicEwoUHHsXh1Ow0z

#### google drive 연결

In [8]:
if IS_COLAB == True:
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/02_o-vton/"

KeyboardInterrupt: ignored

#### 관련 모듈 불러오기 

#### PATH

In [ ]:
if IS_COLAB == True:
  sys.path.append("/content/drive/My Drive/02_o-vton/source/appearance_generation")
  sys.path.append("/content/o-vton/appearance_generation")


In [ ]:
from options.train_options_colab import TrainOptionsColab
from util.visualizer import Visualizer
import util.util as util
from models.models import create_model
from data.ov_train_dataset import RegularDataset
import time
import os
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
from collections import OrderedDict
from subprocess import call
import fractions
from torch.utils.tensorboard import SummaryWriter
from multiprocessing import Process
import matplotlib

%matplotlib inline

#### chekcpoint경로 확인

In [ ]:
!ls /content/drive/MyDrive/02_o-vton/source/appearance_generation/checkpoints

#### main

In [ ]:
def lcm(a, b): return abs(a * b)/fractions.gcd(a, b) if a and b else 0

def main():
  trainOption = TrainOptionsColab()
  trainOption.initialize()

  opt = trainOption.parser

  opt.print_freq = lcm(opt.print_freq, opt.batchSize)
  if opt.debug:
      opt.display_freq = 1
      opt.print_freq = 1
      opt.niter = 1
      opt.niter_decay = 0
      opt.max_dataset_size = 10

  opt.checkpoints_dir = "/content/drive/My Drive/02_o-vton/source/appearance_generation/checkpoints"
  opt.dataroot = "./dataset"
  opt.continue_train = False #이전 checkpoint부터 학습을 하고 싶으면 True.
  opt.gpu_ids = "0"
  opt.nThreads = 0

  iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')
  if opt.continue_train:
      try:
          start_epoch, epoch_iter = np.loadtxt(
              iter_path, delimiter=',', dtype=int)
      except:
          start_epoch, epoch_iter = 1, 0
      print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))
  else:
      start_epoch, epoch_iter = 1, 0



  # NEW DATALOADER
  augment = {}
  augment['1'] = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize((0.5, ), (0.5, ))])

  # augment['1'] = transforms.Compose(
  #     [
  #         transforms.ToTensor()])

  train_dataset = RegularDataset(opt, augment)

  train_dataloader = DataLoader(train_dataset,
                                batch_size=opt.batchSize,
                                shuffle=True,
                                num_workers=int(opt.nThreads),
                                pin_memory=True)

  # FOR DEBUGGING
  print(" #Checking  the dimension and type of data")
  for key in train_dataset[0].keys():
      try:
          x = train_dataset[0][key]
          print("name of the input and shape -- > ", key, x.shape)
          print("type,dtype,and min max -- >", type(x),
                x.dtype, torch.min(x), torch.max(x))
      except Exception as e:
          print("name of the input -- > ", key, train_dataset[0][key])
      print('----------------')

  dataset_size = len(train_dataset)
  print('#training images = %d' % dataset_size)

  # Initialize Networks
  model = create_model(opt)

  # Training Visualizer
  visualizer = Visualizer(opt)

  # Optimizers
  optimizer_G, optimizer_D = model.module.optimizer_G, model.module.optimizer_D

  # Train related additional details
  total_steps = (start_epoch-1) * dataset_size + epoch_iter
  display_delta = total_steps % opt.display_freq
  print_delta = total_steps % opt.print_freq
  save_delta = total_steps % opt.save_latest_freq

  for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
      epoch_start_time = time.time()
      if epoch != start_epoch:
          epoch_iter = epoch_iter % dataset_size
      for i, data in enumerate(train_dataloader, start=epoch_iter):
          if total_steps % opt.print_freq == print_delta:
              iter_start_time = time.time()
          total_steps += opt.batchSize
          epoch_iter += opt.batchSize

          # whether to collect output images
          save_fake = total_steps % opt.display_freq == display_delta

          ############## Forward Pass ######################
          # Reference Purpose
          # input_dict = {'seg_map': A_tensor, 'dense_map': dense_img, 'target': B_tensor, 'seg_map_path': A_path,
          # 'target_path': A_path, 'densepose_path': dense_path }
          # print( data['seg_mask'].shape)
          losses, generated = model(
              data['seg_map'], data['target'], data['seg_mask'], infer=save_fake)

          # sum per device losses
          losses = [torch.mean(x) if not isinstance(x, int)
                    else x for x in losses]
          loss_dict = dict(zip(model.module.loss_names, losses))

          # calculate final loss scalar
          loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
          loss_G = loss_dict['G_GAN'] + loss_dict.get('G_VGG', 0)

          ############### Backward Pass ####################
          # update generator weights
          optimizer_G.zero_grad()
          loss_G.backward()
          optimizer_G.step()

          # update discriminator weights
          optimizer_D.zero_grad()
          loss_D.backward()
          optimizer_D.step()

          ############## Display results and errors ##########
          # print out errors
          if total_steps % opt.print_freq == print_delta:
              errors = {k: v.data.item() if not isinstance(
                  v, int) else v for k, v in loss_dict.items()}
              t = (time.time() - iter_start_time) / opt.print_freq
              visualizer.print_current_errors(epoch, epoch_iter, errors, t)
              visualizer.plot_current_errors(errors, total_steps)
              #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"])

          # display output images
          if save_fake:
              visuals = OrderedDict([('input_label', util.tensor2label(data['seg_map'][0], opt.label_nc)),
                                    ('synthesized_image', util.tensor2im(
                                        generated.data[0])),
                                    ('real_image', util.tensor2im(data['target'][0]))])
              visualizer.display_current_results(visuals, epoch, total_steps)

          # save latest model
          if total_steps % opt.save_latest_freq == save_delta:
              print('saving the latest model (epoch %d, total_steps %d)' %
                    (epoch, total_steps))
              model.module.save('latest')
              np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

          if epoch_iter >= dataset_size:
              break

      # end of epoch
      iter_end_time = time.time()
      print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

      # save model for this epoch
      if epoch % opt.save_epoch_freq == 0:
          print('saving the model at the end of epoch %d, iters %d' %
                (epoch, total_steps))
          model.module.save('latest')
          model.module.save(epoch)
          np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

  #     ### instead of only training the local enhancer, train the entire network after certain iterations
  #     if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
  #         model.module.update_fixed_params()

      # linearly decay learning rate after certain iterations
      if epoch > opt.niter:
          model.module.update_learning_rate()
if __name__ == '__main__':
    # torch.multiprocessing.set_start_method('spawn')
    main()